In [48]:
import tiktoken
import pandas as pd

In [49]:
dataset_path_hf_full="dataset/vampire-bats-6-files/vampire-bats-6-hf.full.jsonl"
questions_per_chunk = 10

In [56]:
df = pd.read_json(dataset_path_hf_full, lines=True)
df.head(2)

,id,type,question,context,oracle_context,cot_answer,instruction
0,3133bb21-2729-47d0-8d22-3dfb24cb1684,general,What is carried by some kitesurfers in a waterproof pouch for emergency use?,"{'sentences': [['Airton Cozzolino holds the record for strapless hangtime at 19 seconds.[20]French kitesurfer Sébastien Cattelan became the first sailor to break the50 knots barrier by reaching 50.26 knots on 3 October 2008 at theLüderitz Speed Challenge in Namibia.[21] On 4 October, Alex Caizergues(also of France) broke this record with a 50.57 knots run.', '1972.', 'Some kitesurfers carrya mobile phone or two-way radio in a waterproof pouch to use in an emergency. A smallEmergency Position-Indicating Radio Beacon (EPIRB) can be carried and activated to send out adistress signal.A buddy is important to help with launching and retrieving the kite, and to assist in anemergency.[59]A GPS can be used to measure distance travelled, tracks and speed during a session.[60]Poncho towels and changing robes are commonly used to dry off and get changed into and outof a wetsuit or swimwear waterside. Poncho towels are more commonly used in warmer weather,whereas changing robes typically have a waterproof shell and provide more insulation for coldertemperatures.Kiteboarding is seen as a mid to high intensity exercise, but freeriding can be a low intensity practicelike walking, and is usually done in long sessions of up to 2–3 hours.', 'On January 13, 2010, Hamilton andwife Reece appeared as themselves on the episode ""Gary Feels Tom Slipping Away"" of the CBStelevision series Gary Unmarried.He was a special guest star as himself in the animated television show Phineas and Ferb. He was alsointerviewed as part of the Australian documentary Bra Boys: Blood is Thicker than Water.Hamilton had a minor role as Troy in The Descendants.Hamilton also was the Celebrity on the Volvo Ocean Race Boat Puma Powered by Berg.']], 'title': [['placeholder_title', 'placeholder_title', 'placeholder_title', 'placeholder_title']]}","Some kitesurfers carrya mobile phone or two-way radio in a waterproof pouch to use in an emergency. A smallEmergency Position-Indicating Radio Beacon (EPIRB) can be carried and activated to send out adistress signal.A buddy is important to help with launching and retrieving the kite, and to assist in anemergency.[59]A GPS can be used to measure distance travelled, tracks and speed during a session.[60]Poncho towels and changing robes are commonly used to dry off and get changed into and outof a wetsuit or swimwear waterside. Poncho towels are more commonly used in warmer weather,whereas changing robes typically have a waterproof shell and provide more insulation for coldertemperatures.Kiteboarding is seen as a mid to high intensity exercise, but freeriding can be a low intensity practicelike walking, and is usually done in long sessions of up to 2–3 hours.","To answer the question, we need to identify what is carried by some kitesurfers in a waterproof pouch for emergency use.\n\nThe context provided gives us the necessary information to determine this.\n\nFirst, we look for the part of the context that directly mentions what is carried by kitesurfers in a waterproof pouch. \nWe find it in the sentence: ##begin_quote##Some kitesurfers carry a mobile phone or two-way radio in a waterproof pouch to use in an emergency. A small Emergency Position-Indicating Radio Beacon (EPIRB) can be carried and activated to send out a distress signal.##end_quote##.\n\nFrom this sentence, we understand that kitesurfers carry a mobile phone, two-way radio, or a small EPIRB in a waterproof pouch for emergency use.\n\nTherefore, based on the context provided, we can conclude that a mobile phone, two-way radio, or a small EPIRB is carried by some kitesurfers in a waterproof pouch for emergency use.\n\n<ANSWER>: A mobile phone, two-way radio, or a small EPIRB.","<DOCUMENT>Airton Cozzolino holds the record for strapless hangtime at 19 seconds.[20]French kitesurfer Sébastien

In [51]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 0)
df.iloc[0]

id                3133bb21-2729-47d0-8d22-3dfb24cb1684                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [52]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    if not string:
        return 0
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [53]:
import sys
import os

sys.path.append(os.path.join(os.getcwd(), os.pardir))

from raft import build_qa_messages

In [54]:
from json import dumps as json_dumps
constant_tokens_in = num_tokens_from_string(json_dumps(build_qa_messages['llama']("", 0))) / questions_per_chunk
constant_tokens_in

19.2

In [57]:
costs_1000 = {
    "in": 0.00378,
    "out": 0.01134,
}
df['tokens_in'] = df['instruction'].apply(num_tokens_from_string) + constant_tokens_in
df['tokens_question'] = df['question'].apply(num_tokens_from_string)
df['tokens_cot_answer'] = df['cot_answer'].apply(num_tokens_from_string)
df['tokens_out'] = df['question'].apply(num_tokens_from_string) + df['cot_answer'].apply(num_tokens_from_string)
df['cost'] = (df['tokens_in'] * costs_1000['in'] + df['tokens_out'] * costs_1000['out']) / 1000
df.head(2)

,id,type,question,context,oracle_context,cot_answer,instruction,tokens_in,tokens_question,tokens_cot_answer,tokens_out,cost
0,3133bb21-2729-47d0-8d22-3dfb24cb1684,general,What is carried by some kitesurfers in a waterproof pouch for emergency use?,"{'sentences': [['Airton Cozzolino holds the record for strapless hangtime at 19 seconds.[20]French kitesurfer Sébastien Cattelan became the first sailor to break the50 knots barrier by reaching 50.26 knots on 3 October 2008 at theLüderitz Speed Challenge in Namibia.[21] On 4 October, Alex Caizergues(also of France) broke this record with a 50.57 knots run.', '1972.', 'Some kitesurfers carrya mobile phone or two-way radio in a waterproof pouch to use in an emergency. A smallEmergency Position-Indicating Radio Beacon (EPIRB) can be carried and activated to send out adistress signal.A buddy is important to help with launching and retrieving the kite, and to assist in anemergency.[59]A GPS can be used to measure distance travelled, tracks and speed during a session.[60]Poncho towels and changing robes are commonly used to dry off and get changed into and outof a wetsuit or swimwear waterside. Poncho towels are more commonly used in warmer weather,whereas changing robes typically have a waterproof shell and provide more insulation for coldertemperatures.Kiteboarding is seen as a mid to high intensity exercise, but freeriding can be a low intensity practicelike walking, and is usually done in long sessions of up to 2–3 hours.', 'On January 13, 2010, Hamilton andwife Reece appeared as themselves on the episode ""Gary Feels Tom Slipping Away"" of the CBStelevision series Gary Unmarried.He was a special guest star as himself in the animated television show Phineas and Ferb. He was alsointerviewed as part of the Australian documentary Bra Boys: Blood is Thicker than Water.Hamilton had a minor role as Troy in The Descendants.Hamilton also was the Celebrity on the Volvo Ocean Race Boat Puma Powered by Berg.']], 'title': [['placeholder_title', 'placeholder_title', 'placeholder_title', 'placeholder_title']]}","Some kitesurfers carrya mobile phone or two-way radio in a waterproof pouch to use in an emergency. A smallEmergency Position-Indicating Radio Beacon (EPIRB) can be carried and activated to send out adistress signal.A buddy is important to help with launching and retrieving the kite, and to assist in anemergency.[59]A GPS can be used to measure distance travelled, tracks and speed during a session.[60]Poncho towels and changing robes are commonly used to dry off and get changed into and outof a wetsuit or swimwear waterside. Poncho towels are more commonly used in warmer weather,whereas changing robes typically have a waterproof shell and provide more insulation for coldertemperatures.Kiteboarding is seen as a mid to high intensity exercise, but freeriding can be a low intensity practicelike walking, and is usually done in long sessions of up to 2–3 hours.","To answer the question, we need to identify what is carried by some kitesurfers in a waterproof pouch for emergency use.\n\nThe context provided gives us the necessary information to determine this.\n\nFirst, we look for the part of the context that directly mentions what is carried by kitesurfers in a waterproof pouch. \nWe find it in the sentence: ##begin_quote##Some kitesurfers carry a mobile phone or two-way radio in a waterproof pouch to use in an emergency. A small Emergency Position-Indicating Radio Beacon (EPIRB) can be carried and activated to send out a distress signal.##end_quote##.\n\nFrom this sentence, we understand that kitesurfers carry a mobile phone, two-way radio, or a small EPIRB in a waterproof pouch for emergency use.\n\nTherefore, based on the context provided, we can conclude that a mobile phone, two-way radio, or a small EPIRB is carried by some kitesurfers in a waterproof pouch for emergency use.\n\n<ANSWER>: A mobile phone, two-way radio, or a small EPIRB.","<DOCUMENT>Airton Cozzolino holds the record for stra

In [ ]:
df[['tokens_in', 'tokens_out', 'cost']].sum()

tokens_in     1.011737e+07
tokens_out    3.546151e+06
cost          7.845701e+01
dtype: float64

In [ ]:
len(df)

18666